# Implementing Recommendation Engines with Surprise

## Introduction

This lesson will give you a brief introduction to implementing recommendation engines with a Python library called `surprise`. You'll get a chance to try out multiple different types of collaborative filtering engines, ranging from both basic neighborhood-based methods to matrix factorization methods. 

## Objectives
You will be able to:

- Use `surprise`'s built-in reader class to process data to work with recommender algorithms 
- Use `surprise` to create and cross-validate different recommender algorithms 
- Obtain a prediction for a specific user for a particular item 

In this lesson, we'll be working with a dataset built-in to `surprise` called the `jester` dataset. This dataset contains jokes rated by users on a scale of -10 to 10 based off a user's perceived humor level for a given joke. Let's get recommending some jokes! 

First, you'll have to load the jokes dataset. You might have to download it first if prompted. Let's investigate the dataset after we load that. In this folder, you'll find the file for the text of jokes if you want to investigate what caliber of human you're dealing with here.

In [3]:
# here are the relevant library to work with..
# from surprise import Dataset
# from surprise.model_selection import train_test_split

In [4]:
! pip install scikit surprise

ERROR: Could not find a version that satisfies the requirement scikit (from versions: none)
ERROR: No matching distribution found for scikit


In [5]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

jokes = Dataset.load_builtin(name="jester")
jokes

In [6]:
type(jokes)

surprise.dataset.DatasetAutoFolds

In [7]:
# Split into train and test set
trainset, testset = train_test_split(jokes, test_size=0.2)

In [8]:
# lets get access to the content on the Train set
# check for the number of users and items 
num_users = trainset.n_users
num_items = trainset.n_items

print("Here are the number of users for each item ", num_users)
print("Here are the number of items for each user", num_items)

Here are the number of users for each item  58782
Here are the number of items for each user 140


Notice how there is no `X_train` or `y_train` in our values here. Our only features here are the ratings of other users and items, so we need to keep everything together. What is happening in the train-test split here is that `surprise` is randomly selecting certain $r_{ij}$ for users $u_{i}$ and items $i_{j} $. 80% of the ratings are in the training set and 20% in the test set. Let's investigate `trainset` and `testset` further. 

In [9]:
print("Type trainset :", type(trainset), "\n") # surprise randomly selects certain users ui and items ij
print("Type testset :", type(testset))

Type trainset : <class 'surprise.trainset.Trainset'> 

Type testset : <class 'list'>


Interestingly enough, the values here are of different data types! The `trainset` is still a `surprise` specific data type that is optimized for computational efficiency and the `testset` is a standard Python list - you'll see why when we start making predictions. Let's take a look at how large our `testset` is as well as what's contained in an individual element. A sacrifice of `surprise`'s implementation is that we lose a lot of the exploratory methods that are present with Pandas.

In [10]:
print(len(testset))

print(testset[0])

# print(len(testset))
# print(testset[0])

352288
('7779', '89', 2.469)


## Memory-Based Methods (Neighborhood-Based)

To begin with, we can calculate the more simple neighborhood-based approaches. Some things to keep in mind are what type of similarities you should use. These can all have fairly substantial effects on the overall performance of the model. You'll notice that the API of surprise is very similar to scikit-learn when it comes to model fitting and testing. To begin with, we'll import the modules we'll be using for the neighborhood-based methods.

In [11]:
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
import numpy as np
np.int = int

One of our first decisions is item-item similarity versus user-user similarity. For the sake of computation time, it's best to calculate the similarity between whichever number is fewer, users or items. Let's see what the case is for our training set.

In [12]:
print("Number of users: ", trainset.n_users, "\n")
print("Number of items: ", trainset.n_items, "\n")

# there's way more users than items, since there's fewer items than user its more efficient to calculate item-item similarity rather than user-user
# similarity

Number of users:  58782 

Number of items:  140 



There are clearly way more users than items! We'll take that into account when inputting the specifications to our similarity metrics. Because we have fewer items than users, it will be more efficient to calculate item-item similarity rather than user-user similarity.

In [13]:
# get the cosine similarity
sim_cosine = {'name': 'cosine', 'user_based':  False}
print(sim_cosine)
# sim_cos = {"name": "cosine", "user_based": False}

{'name': 'cosine', 'user_based': False}


Now it's time to train our model. Note that if you decide to train this model with `user_based=True`, it will take quite some time!

In [14]:
# lets fit the sim_cosine on the train data using the user_based = False.
# by using the True the training would take a long time to train

base_knn = knns.KNNBasic(sim_options = sim_cosine)
base_knn.fit(trainset)

# basic = knns.KNNBasic(sim_options=sim_cosine)
# basic.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


And now our model is fit! Let's take a look at the similarity metrics of each of the items to one another by using the `sim` attribute of our fitted model.

In [15]:
base_knn.sim

# basic.sim

array([[ 1.        , -0.0342802 ,  0.64296905, ...,  0.51702641,
         0.42169253,  0.14473167],
       [-0.0342802 ,  1.        , -0.02909392, ..., -0.13527914,
        -0.06980483, -0.05806708],
       [ 0.64296905, -0.02909392,  1.        , ...,  0.70618388,
         0.59187372,  0.40058678],
       ...,
       [ 0.51702641, -0.13527914,  0.70618388, ...,  1.        ,
         0.57118637,  0.39254659],
       [ 0.42169253, -0.06980483,  0.59187372, ...,  0.57118637,
         1.        ,  0.49216551],
       [ 0.14473167, -0.05806708,  0.40058678, ...,  0.39254659,
         0.49216551,  1.        ]])

Now it's time to test the model to determine how well it performed.

In [16]:
# test the model performance..
preds = base_knn.test(testset)
preds


# predictions = basic.test(testset)

[Prediction(uid='7779', iid='89', r_ui=2.469, est=3.8085283206015688, details={'actual_k': 25, 'was_impossible': False}),
 Prediction(uid='992', iid='8', r_ui=-0.688, est=0.9075645428674513, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='36748', iid='17', r_ui=2.438, est=3.739647801485279, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='54820', iid='101', r_ui=-9.156, est=-3.190701253511137, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='14902', iid='115', r_ui=9.719, est=4.028341136565893, details={'actual_k': 17, 'was_impossible': False}),
 Prediction(uid='47474', iid='146', r_ui=0.844, est=-1.4227682894820823, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='40320', iid='143', r_ui=8.562, est=8.29464399130006, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='61050', iid='148', r_ui=-2.219, est=2.553740253899448, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=

In [ ]:
# check for the accuracy score
print(accuracy.rmse(preds))

# the model has rmse value of 4.2165 which ,means that its off by roughly 4 points for each random guess it made for the rating

# print(accuracy.rmse(predictions))

RMSE: 4.2165
4.216494349912535


Not a particularly amazing model.... As you can see, the model had an RMSE of about 4.5, meaning that it was off by roughly 4 points for each guess it made for ratings. Not horrendous when you consider we're working on a range of 20 points, but let's see if we can improve it. To begin with, let's try with a different similarity metric (Pearson correlation) and evaluate our RMSE.

In [27]:
# use the pearson similarity then evaluate its performance..
sim_pearson = {'name': 'pearson', 'user_based': False}
base_knn_pearson = knns.KNNBasic(sim_options = sim_pearson)

# fit the model on the test set
base_knn_pearson.fit(trainset)

# get the predictions
pred_1 = base_knn_pearson.test(testset)

# evaluate the accuracy score on the pearson similarity..
print(accuracy.rmse(pred_1))

# the pearson similarity has performed better than the cosine similarity in the base KNN mode

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 4.2775
4.2775298117369545


In [18]:
# sim_pearson = {"name": "pearson", "user_based": False}
# basic_pearson = knns.KNNBasic(sim_options=sim_pearson)
# basic_pearson.fit(trainset)
# predictions = basic_pearson.test(testset)
# print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 4.2775
4.2775298117369545


Pearson correlation seems to have performed better than cosine similarity in the basic KNN model, we can go ahead and use Pearson correlation as our similarity metric of choice for future models. The next model we're going to try is [KNN with Means](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNWithMeans). This is the same thing as the basic KNN model, except it takes into account the mean rating of each user or item depending on whether you are performing user-user or item-item similarities, respectively.

In [28]:
# lets try using the KNN with means which involves taking into account the mean rating of each user or item depending on whether 
# you're performing user-user or item-item similarities..
sim_pearson = {'name': 'pearson', 'user_based': False}
knn_mean = knns.KNNWithMeans(sim_options = sim_pearson)

# fitting the model
knn_mean.fit(trainset)
pred_2 = knn_mean.test(testset)

print(accuracy.rmse(pred_2))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 4.1365
4.136476380648139


In [29]:
# sim_pearson = {"name": "pearson", "user_based": False}
# knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
# knn_means.fit(trainset)
# predictions = knn_means.test(testset)
# print(accuracy.rmse(predictions))

A little better... let's try one more neighborhood-based method before moving into model-based methods. Let's try the [KNNBaseline](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNBaseline) method. This is a more advanced method because it adds in a bias term that is calculated by way of minimizing a cost function represented by:

$$ \sum_{r_{ui} \in R_{\text{train}}}{(\hat{r}_{ui} - ( \mu + b_{i} + b_{u}))^{2} + \lambda(b_u^2 + b_i^2) } $$

With $b_i$ and $b_u$ being biases for items and users respectively and $\mu$ referring to the global mean.

In [20]:
sim_pearson = {"name": "pearson", "user_based": False}
knn_baseline = knns.KNNBaseline(sim_options=sim_pearson)
knn_baseline.fit(trainset)
predictions = knn_baseline.test(testset)
print(accuracy.rmse(predictions))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 4.1322
4.132202989811008


Even better! Now let's see if we can get some insight by applying some matrix factorization techniques!

## Model-Based Methods (Matrix Factorization)

It's worth pointing out that when SVD is calculated for recommendation systems, it is preferred to be done with a modified version called "Funk's SVD" that only takes into account the rated values, ignoring whatever items have not been rated by users. The algorithm is named after Simon Funk, who was part of the team who placed 3rd in the Netflix challenge with this innovative way of performing matrix decomposition. Read more about Funk's SVD implementation at [his original blog post](https://sifter.org/~simon/journal/20061211.html). There is no simple way to include for this fact with SciPy's implementation of `svd()`, but luckily the `surprise` library has Funk's version of SVD implemented to make our lives easier!

Similar to other `sklearn` features, we can expedite the process of trying out different parameters by using an implementation of grid search. Let's make use of the grid search here to account for some different configurations of parameters within the SVD pipeline. This might take some time! You'll notice that the `n_jobs` is parameter set to -1, which ensures that all of the cores on your computer will be used to process fitting and evaluating all of these models. To help keep track of what is occurring here, take note of the different values. This code ended up taking over 16 minutes to complete even with parallelization in effect, so the optimal parameters are given to you for the SVD model below. Use them to train a model and let's see how well it performs. If you want the full grid search experience, feel free to uncomment the code and give it a go!

The optimal parameters are :

```python
{'n_factors': 100, 'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}

```

In [30]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

# param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
# gs_model.fit(jokes)

In [31]:
svd = SVD(n_factors = 100, n_epochs = 10, lr_all = 0.005, reg_all = 0.4)

# fit the algorithm on the test set..
svd.fit(trainset)

# get the predictions
pred_3 = svd.test(testset)
print(accuracy.rmse(pred_3))

RMSE: 4.2774
4.277398789443669


In [22]:
# svd = SVD(n_factors=100, n_epochs=10, lr_all=0.005, reg_all=0.4)
# svd.fit(trainset)
# predictions = svd.test(testset)
# print(accuracy.rmse(predictions))

RMSE: 4.2704
4.270431174267582


Interestingly, this model performed worse than the others! In general, the advantages of matrix factorization starts to show itself when the size of the dataset becomes massive. At that point, the storage challenges increase for the memory-based models, and there is enough data for latent factors to become extremely apparent.

## Making Predictions

Now that we've explored some models, we can think about how we might fit the models into the context of an application. To begin with, let's access some basic functionality of `surprise` models to get predicted ratings for a given user and item. All that's needed are the `user_id` and `item_id` for which you want to make a prediction. Here we're making a prediction for user 34 and item 25 using the SVD model we just fit. 

In [23]:
user_34_prediction = svd.predict("34", "25")
user_34_prediction

Prediction(uid='34', iid='25', r_ui=None, est=2.204309697654434, details={'was_impossible': False})

The output of the prediction is a tuple. Here, we're going to access the estimated rating.

In [24]:
user_34_prediction[3]

2.204309697654434

You might be wondering, "OK I'm making predictions about certain items rated by certain users, but how can I actually give certain N recommendations to an individual user?" Although `surprise`  is a great library, it does not have this recommendation functionality built into it, but in the next lab, you will get some experience not only fitting recommendation system models, but also programmatically retrieving recommended items for each user.

### Sources


Jester dataset originally obtained from:

[Eigentaste](http://www.ieor.berkeley.edu/~goldberg/pubs/eigentaste.pdf): A Constant Time Collaborative Filtering Algorithm. Ken Goldberg, Theresa Roeder, Dhruv Gupta, and Chris Perkins. Information Retrieval, 4(2), 133-151. July 2001.



### Additional Resources

- [Surprise Documentation](https://surprise.readthedocs.io/en/stable/index.html) 
- [Surprise Tutorial](https://blog.cambridgespark.com/tutorial-practical-introduction-to-recommender-systems-dbe22848392b)

## Summary

You now should have an understanding of the basic considerations one should take note of when coding a recommendation system as well as how to implement them in different ways using `surprise`. In the upcoming lab, you will be tasked with fitting models using `surprise` and then retrieving those predicted values in a meaningful way to give recommendations to people. Let's see how well it works in action.